
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
- Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

- Cargaremos los dos ficheros de datos

In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast

df_clima= pd.read_pickle("../Ficheros/ETL1.pkl")
df_attacks= pd.read_csv("../Ficheros/attacks_limpieza_completa.csv", index_col = 0)

In [2]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea

In [3]:
df_seleccion = df_attacks[(df_attacks["country"]== "usa") | (df_attacks["country"] == "australia") | (df_attacks["country"] == "new zealand") | (df_attacks["country"] == "south africa") | (df_attacks["country"] == "papua new guinea")] 

In [4]:
df_seleccion.country.unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [5]:
df_seleccion.sample(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
40,2018,Provoked,australia,34.0,Unespecific,Feb,N,M,-24.776109,134.755,HS,1,0,0,1,0,0,3,4,4,7.099693
6205,0,Unprovoked,australia,27.0,Unespecific,Unknown,Unknown,M,-24.776109,134.755,HS,0,1,0,0,1,0,3,14,7,6.313925



- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [6]:
#df_clima['rh_profile'] = df_clima['rh_profile'].apply(ast.literal_eval)

In [7]:
df_clima['rh_profile'].iloc[0]

[{'layer': '950mb', 'rh': 9},
 {'layer': '900mb', 'rh': 9},
 {'layer': '850mb', 'rh': 10},
 {'layer': '800mb', 'rh': 10},
 {'layer': '750mb', 'rh': 9},
 {'layer': '700mb', 'rh': 8},
 {'layer': '650mb', 'rh': 5},
 {'layer': '600mb', 'rh': 3},
 {'layer': '550mb', 'rh': 3},
 {'layer': '500mb', 'rh': 2},
 {'layer': '450mb', 'rh': 1},
 {'layer': '400mb', 'rh': 0},
 {'layer': '350mb', 'rh': 2},
 {'layer': '300mb', 'rh': 5},
 {'layer': '250mb', 'rh': 7},
 {'layer': '200mb', 'rh': 3}]

In [8]:
x = df_clima['rh_profile'].apply(pd.Series)

for values in range(len(x.columns)):
    nombre='rh' + str(x[values].apply(pd.Series)['layer'][0])
    valores = list(x[values].apply(pd.Series)['rh'])
    df_clima.insert(values, nombre, valores)

In [9]:
df_clima['wind_profile'].iloc[0]

[{'layer': '950mb', 'direction': 175, 'speed': 2},
 {'layer': '900mb', 'direction': 225, 'speed': 2},
 {'layer': '850mb', 'direction': 260, 'speed': 3},
 {'layer': '800mb', 'direction': 290, 'speed': 3},
 {'layer': '750mb', 'direction': 300, 'speed': 3},
 {'layer': '700mb', 'direction': 305, 'speed': 3},
 {'layer': '650mb', 'direction': 315, 'speed': 3},
 {'layer': '600mb', 'direction': 320, 'speed': 2},
 {'layer': '550mb', 'direction': 320, 'speed': 3},
 {'layer': '500mb', 'direction': 305, 'speed': 3},
 {'layer': '450mb', 'direction': 300, 'speed': 4},
 {'layer': '400mb', 'direction': 290, 'speed': 4},
 {'layer': '350mb', 'direction': 280, 'speed': 4},
 {'layer': '300mb', 'direction': 285, 'speed': 5},
 {'layer': '250mb', 'direction': 280, 'speed': 5},
 {'layer': '200mb', 'direction': 280, 'speed': 5}]

In [10]:
y = df_clima['wind_profile'].apply(pd.Series)

for values in range(len(y.columns)):
    nombre='wind_speed' + str(y[values].apply(pd.Series)['layer'][0])
    valores = list(y[values].apply(pd.Series)['speed'])
    df_clima.insert(values, nombre, valores)

In [11]:
df_clima['paises'] = df_clima['paises'].apply(lambda x:x.lower())

In [12]:
df_clima.head(1)

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,2,2,3,3,3,3,3,2,3,3,4,4,4,5,5,5,9,9,10,10,9,8,5,3,3,2,1,0,2,5,7,3,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 175, 'speed':...",26,2,6,1011,none,0,0,130,2,39.78373,-100.445882,usa


In [13]:
df_clima_final = df_clima.groupby("paises").mean().reset_index()
df_clima_final

,paises,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,latitud,longitud
0,australia,3.062500,2.875000,2.781250,2.750000,2.859375,2.828125,2.812500,2.906250,3.109375,3.531250,4.296875,4.484375,4.859375,5.656250,6.546875,7.031250,13.593750,11.218750,5.234375,2.375000,0.875000,1.078125,1.046875,-0.640625,-1.750000,-0.484375,0.281250,0.218750,0.937500,2.468750,5.734375,1.765625,97.5,3.500000,-9999.0,-9999.0,-9999.0,21.906250,2.484375,10.281250,1022.921875,2.125000,0.0,2.937500,-24.776109,134.755000
1,new zealand,3.937500,3.828125,3.718750,3.718750,3.671875,3.812500,3.812500,3.921875,4.125000,4.328125,4.531250,4.671875,5.093750,5.421875,5.875000,5.718750,11.750000,12.796875,7.109375,4.125000,3.000000,0.609375,1.234375,2.046875,2.687500,4.000000,3.734375,4.765625,6.234375,5.859375,6.609375,-0.515625,97.5,7.734375,-9999.0,-9999.0,-9999.0,11.062500,8.125000,10.687500,1024.390625,2.046875,0.0,3.406250,-41.500083,172.834408
2,papua new guinea,4.093750,4.125000,3.765625,3.125000,2.625000,2.765625,2.734375,2.796875,3.187500,3.234375,2.890625,3.093750,3.531250,4.218750,4.828125,5.140625,13.343750,11.109375,6.984375,3.234375,0.859375,-0.718750,-2.812500,-3.562500,-3.875000,-3.953125,-3.187500,-2.546875,-2.218750,-1.531250,-1.062500,-1.171875,97.5,2.265625,-9999.0,-9999.0,-9999.0,25.781250,2.218750,10.906250,1011.312500,1.937500,0.0,3.515625,-5.681607,144.248908
3,south africa,3.546875,3.609375,3.562500,3.703125,3.984375,4.109375,4.515625,5.000000,5.625000,6.328125,6.734375,7.281250,7.828125,8.984375,10.046875,10.328125,12.046875,11.468750,10.531250,9.531250,9.093750,7.687500,5.156250,3.890625,4.218750,6.218750,8.375000,9.359375,10.328125,8.031250,6.296875,3.937500,97.5,7.156250,-9999.0,-9999.0,-9999.0,17.921875,1.703125,12.296875,1014.781250,4.437500,0.0,3.187500,-28.816624,24.991639
4,usa,2.265625,2.187500,2.203125,2.421875,2.828125,3.031250,3.171875,3.312500,3.625000,3.718750,3.906250,4.296875,4.593750,4.984375,5.343750,5.640625,11.187500,12.843750,10.984375,9.546875,9.109375,8.703125,7.625000,5.796875,4.484375,3.671875,3.703125,5.078125,6.453125,7.843750,9.578125,5.140625,97.5,5.250000,-9999.0,-9999.0,-9999.0,24.140625,-0.687500,8.140625,1014.187500,2.734375,0.0,2.296875,39.783730,-100.445882


- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

In [14]:
df_seleccion.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


In [15]:
df_clima_final.rename(columns={"paises":"country"}, inplace=True)

In [16]:
df_final = pd.merge(left = df_seleccion, right= df_clima_final, how= "left", on= "country")
df_final.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud_x,longitud_x,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,latitud_y,longitud_y
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327,2.265625,2.1875,2.203125,2.421875,2.828125,3.031250,3.171875,3.31250,3.625000,3.71875,3.906250,4.296875,4.593750,4.984375,5.343750,5.640625,11.18750,12.84375,10.984375,9.546875,9.109375,8.703125,7.625000,5.796875,4.484375,3.671875,3.703125,5.078125,6.453125,7.84375,9.578125,5.140625,97.5,5.25,-9999.0,-9999.0,-9999.0,24.140625,-0.687500,8.140625,1014.187500,2.734375,0.0,2.296875,39.783730,-100.445882
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860,2.265625,2.1875,2.203125,2.421875,2.828125,3.031250,3.171875,3.31250,3.625000,3.71875,3.906250,4.296875,4.593750,4.984375,5.343750,5.640625,11.18750,12.84375,10.984375,9.546875,9.109375,8.703125,7.625000,5.796875,4.484375,3.671875,3.703125,5.078125,6.453125,7.84375,9.578125,5.140625,97.5,5.25,-9999.0,-9999.0,-9999.0,24.140625,-0.687500,8.140625,1014.187500,2.734375,0.0,2.296875,39.783730,-100.445882
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629,2.265625,2.1875,2.203125,2.421875,2.828125,3.031250,3.171875,3.31250,3.625000,3.71875,3.906250,4.296875,4.593750,4.984375,5.343750,5.640625,11.18750,12.84375,10.984375,9.546875,9.109375,8.703125,7.625000,5.796875,4.484375,3.671875,3.703125,5.078125,6.453125,7.84375,9.578125,5.140625,97.5,5.25,-9999.0,-9999.0,-9999.0,24.140625,-0.687500,8.140625,1014.187500,2.734375,0.0,2.296875,39.783730,-100.445882
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925,3.062500,2.8750,2.781250,2.750000,2.859375,2.828125,2.812500,2.90625,3.109375,3.53125,4.296875,4.484375,4.859375,5.656250,6.546875,7.031250,13.59375,11.21875,5.234375,2.375000,0.875000,1.078125,1.046875,-0.640625,-1.750000,-0.484375,0.281250,0.218750,0.937500,2.46875,5.734375,1.765625,97.5,3.50,-9999.0,-9999.0,-9999.0,21.906250,2.484375,10.281250,1022.921875,2.125000,0.0,2.937500,-24.776109,134.755000
4,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925,3.062500,2.8750,2.781250,2.750000,2.859375,2.828125,2.812500,2.90625,3.109375,3.53125,4.296875,4.484375,4.859375,5.656250,6.546875,7.031250,13.59375,11.21875,5.234375,2.375000,0.875000,1.078125,1.046875,-0.640625,-1.750000,-0.484375,0.281250,0.218750,0.937500,2.46875,5.734375,1.765625,97.5,3.50,-9999.0,-9999.0,-9999.0,21.906250,2.484375,10.281250,1022.921875,2.125000,0.0,2.937500,-24.776109,134.755000


In [17]:
df_final.to_csv("../Ficheros/ETL2_1.csv")